# Repeval 2017 Exporations

In [1]:
from gensim.models import KeyedVectors

In [2]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.preprocessing import text, sequence
from keras.layers import merge
from keras.utils.np_utils import to_categorical

Using Theano backend.
Using gpu device 0: GeForce GTX 1050 Ti (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5110)
/home/fabian/anaconda3/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


First, load pre-trained word embeddings. Here, the ones from Mikolov using the word2vec toolkit

In [3]:
W2V_BINARY_PATH = '../data/GoogleNews-vectors-negative300.bin'
vectorspace = KeyedVectors.load_word2vec_format(W2V_BINARY_PATH, binary=True)

Load the data. Starting here with the SNLI Corpus until further data are available for the repeval2017 task.

In [4]:
data_path = '../data/snli_1.0/'
data_frame = pd.read_csv(data_path + 'snli_1.0_train.txt', sep='\t')
gold_labels = data_frame.gold_label.tolist()
sentences1 = data_frame.sentence1.tolist()
sentences2 = data_frame.sentence2.tolist()

Use only part of the data for testing the model:

In [5]:
num_samples = 10000
samples = [(sentences1[i], sentences2[i], gold_labels[i]) for i in range(num_samples)]
print("Using {} samples from the dataset".format(len(samples)))

Using 10000 samples from the dataset


## Model Architecture

In [22]:
def conv_block(num_layers, input_first_layer, filters):
    inputs = []
    inputs.append(input_first_layer)
    for i in range(num_layers):
        output_conv = Conv1D(nb_filter=filters, filter_length=3, activation='relu')(inputs[i])
        inputs.append(output_conv)
    output = MaxPooling1D()(inputs[i+1])
    return output

In [26]:
def build_model(input_size = (10,128)):
    inputs = Input(shape=input_size)
    outputs = conv_block(1, inputs, 64)
    
    model = Model(input=inputs, output=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [34]:
#test
model = build_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 10, 128)       0                                            
____________________________________________________________________________________________________
convolution1d_5 (Convolution1D)  (None, 8, 64)         24640       input_8[0][0]                    
____________________________________________________________________________________________________
maxpooling1d_5 (MaxPooling1D)    (None, 4, 64)         0           convolution1d_5[0][0]            
Total params: 24,640
Trainable params: 24,640
Non-trainable params: 0
____________________________________________________________________________________________________


In [25]:
??Model

## Training the model with some data

In [49]:
#todo use word embeddings instead of one-hot
max_length_sentence = np.max([len(sentence.split()) for sentence, sentence2, labels in samples])
print(max_length_sentence)

26


In [51]:
def embed_sentence(sentence, vectorspace=vectorspace):
    result = np.array([])
    missing_words = []
    for word in sentence.split():  #TODO apply better tokenizing
        try:
            result = np.concatenate((result, vectorspace[word]))
        except:
            print("Missing {}".format(word))
            missing_words.append(word)
    #print(np.shape(result))
    #print(len)
    return result

Preprocessing the data to include the word embeddings into the model is done after this keras-example:
https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py

In [6]:
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2

In [7]:
sentences1 = [sentence for sentence, sentence2, label in samples]
sentences2 = [sentence2 for sentence, sentence2, label in samples]
labels = [label for sentence, sentence2, label in samples] 
numeric_labels = []
for label in labels:
    if label == 'neutral':
        numeric_labels.append(0)
    elif label == 'contradiction':
        numeric_labels.append(1)
    elif label == 'entailment':
        numeric_labels.append(2)
    else:
        numeric_labels.append(3)
        
tokenizer = text.Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(sentences1 + sentences2)
sequences1 = tokenizer.texts_to_sequences(sentences1)
sequences2 = tokenizer.texts_to_sequences(sentences2)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data1 = sequence.pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)
data2 = sequence.pad_sequences(sequences2, maxlen=MAX_SEQUENCE_LENGTH)

data = np.asarray([data1, data2])

labels = to_categorical(np.asarray(numeric_labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[1])
np.random.shuffle(indices)
data = data[:,indices,:]
labels = labels[indices,:]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[1])

x_train = data[:,:-num_validation_samples,:]
y_train = labels[:-num_validation_samples]
x_val = data[:,-num_validation_samples:,:]
y_val = labels[-num_validation_samples:]

print("Shape of x_train: {}".format(np.shape(x_train)))
print("Shape of y_train: {}".format(np.shape(y_train)))
print("Shape of x_val: {}".format(np.shape(x_val)))
print("Shape of y_val: {}".format(np.shape(y_val)))

print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words + 1, EMBEDDING_DIM)) #TODO: is this correct? In the example, the first dimension is
# num_words, this, however, throws an error when populating the embedding matrix (because the word-indices start at 1, not at 0)
print("shape of embedding matrix: {}".format(np.shape(embedding_matrix)))

for word, i in word_index.items():
    #print("{}: {}".format(i,word))
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = None
    
    try:
        embedding_vector = vectorspace[word]
    except:
        pass
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False) #set trainable = True to enable training of the embeddings to the task at hand

Found 5843 unique tokens.
Shape of data tensor: (2, 10000, 1000)
Shape of label tensor: (10000, 4)
Shape of x_train: (2, 8000, 1000)
Shape of y_train: (8000, 4)
Shape of x_val: (2, 2000, 1000)
Shape of y_val: (2000, 4)
Preparing embedding matrix.
shape of embedding matrix: (5844, 300)


Todo next:
+ build model architecture that uses the same layers for encoding both sentences (starting with the tutorial used above)
+ build model on top to predict entailment, neutral, contradiction (see intro functional, shared layers)
+ test different architectures

The model for encoding the sentences:

In [8]:
sentence1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
sentence2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

#encode first sentence
embedded_sentence1 = embedding_layer(sentence1_input)
x = Conv1D(128, 5, activation='relu')(embedded_sentence1)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
encoded_sentence1 = MaxPooling1D(35)(x)

#encode second sentence
embedded_sentence2 = embedding_layer(sentence2_input)
y = Conv1D(128, 5, activation='relu')(embedded_sentence2)
y = MaxPooling1D(5)(y)
y = Conv1D(128, 5, activation='relu')(y)
y = MaxPooling1D(5)(y)
y = Conv1D(128, 5, activation='relu')(y)
encoded_sentence2 = MaxPooling1D(35)(y)

#merge the encoded sentences (First: concatenation)
merged_vector = merge(inputs = [encoded_sentence1, encoded_sentence2], mode='concat', concat_axis=-1)

#predict the labels
flat = Flatten()(merged_vector)
x = Dense(256, activation='relu')(flat)
preds = Dense(4,activation='softmax')(x)

#compile the model
model = Model(input=[sentence1_input, sentence2_input], output=preds)
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [32]:
model.fit([x_train[0,:,:], x_train[1,:,:]], y_train, 
          nb_epoch=10, 
          validation_data=([x_val[0,:,:], x_val[1,:,:]], y_val),
          batch)

Train on 24 samples, validate on 6 samples
Epoch 1/10
24/24 [==============================] - 0s - loss: 1.0977 - acc: 0.3750 - val_loss: 1.1004 - val_acc: 0.3333
Epoch 2/10
24/24 [==============================] - 0s - loss: 1.0949 - acc: 0.3750 - val_loss: 1.1054 - val_acc: 0.3333
Epoch 3/10
24/24 [==============================] - 0s - loss: 1.0886 - acc: 0.3750 - val_loss: 1.1180 - val_acc: 0.3333
Epoch 4/10
24/24 [==============================] - 0s - loss: 1.0765 - acc: 0.3750 - val_loss: 1.1489 - val_acc: 0.3333
Epoch 5/10
24/24 [==============================] - 0s - loss: 1.0572 - acc: 0.3750 - val_loss: 1.2161 - val_acc: 0.3333
Epoch 6/10
24/24 [==============================] - 0s - loss: 1.0337 - acc: 0.3750 - val_loss: 1.3344 - val_acc: 0.3333
Epoch 7/10
24/24 [==============================] - 0s - loss: 1.0116 - acc: 0.3750 - val_loss: 1.4287 - val_acc: 0.3333
Epoch 8/10
24/24 [==============================] - 0s - loss: 0.9850 - acc: 0.3750 - val_loss: 1.4522 - val_a

In [9]:
??model.fit
